<p style="margin: 5px 0 0 0; color: #666;"><em>Desarrollado con Claude - Anthropic</em></p>

# 8. SQL y Bases de Datos

In [2]:
import sqlite3
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

print("✓ Librerías cargadas correctamente")
print(f"SQLite versión: {sqlite3.sqlite_version}")
print(f"Pandas versión: {pd.__version__}")

✓ Librerías cargadas correctamente
SQLite versión: 3.42.0
Pandas versión: 3.0.0


## Preparación: Crear Base de Datos de Ejemplo

### ¿Qué es?

La **preparación de una base de datos de ejemplo** es el proceso de crear una base de datos relacional con tablas, relaciones y datos de muestra que sirven como entorno de práctica. En este caso, usamos SQLite, un motor de base de datos ligero integrado en Python, para construir un esquema de ventas con tablas de categorías, productos, clientes y transacciones.

### ¿Para qué sirve?

Crear una base de datos de ejemplo sirve para:

- **Practicar consultas SQL** en un entorno seguro sin afectar datos reales
- **Entender el modelo relacional** viendo cómo las tablas se conectan mediante claves foráneas
- **Simular escenarios reales** de negocio con datos de ventas, clientes y productos
- **Experimentar libremente** con SELECT, JOIN, GROUP BY y otras operaciones
- **Comprender la estructura de datos** antes de trabajar con bases de datos de producción
- **Validar consultas** y verificar resultados con datos conocidos

En análisis de datos, tener un dataset de práctica es esencial para aprender y probar nuevas técnicas antes de aplicarlas a datos reales.

### ¿Cómo se usa?

Se crea una conexión con `sqlite3.connect()`, se definen las tablas con `CREATE TABLE`, se insertan datos con `INSERT INTO`, y se confirman los cambios con `commit()`. A continuación creamos cuatro tablas relacionadas: categorías, productos, clientes y ventas.

In [3]:
print("=" * 70)
print("CREACIÓN DE BASE DE DATOS".center(70))
print("=" * 70)

# Crear conexión a base de datos SQLite
conn = sqlite3.connect('ventas.db')
cursor = conn.cursor()

# Eliminar tablas si existen
cursor.execute('DROP TABLE IF EXISTS ventas')
cursor.execute('DROP TABLE IF EXISTS productos')
cursor.execute('DROP TABLE IF EXISTS clientes')
cursor.execute('DROP TABLE IF EXISTS categorias')

# Tabla de categorías
cursor.execute('''
CREATE TABLE categorias (
    id_categoria INTEGER PRIMARY KEY,
    nombre_categoria TEXT NOT NULL,
    descripcion TEXT
)
''')

# Tabla de productos
cursor.execute('''
CREATE TABLE productos (
    id_producto INTEGER PRIMARY KEY,
    nombre_producto TEXT NOT NULL,
    precio REAL NOT NULL,
    id_categoria INTEGER,
    stock INTEGER,
    FOREIGN KEY (id_categoria) REFERENCES categorias(id_categoria)
)
''')

# Tabla de clientes
cursor.execute('''
CREATE TABLE clientes (
    id_cliente INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    email TEXT UNIQUE,
    ciudad TEXT,
    pais TEXT
)
''')

# Tabla de ventas
cursor.execute('''
CREATE TABLE ventas (
    id_venta INTEGER PRIMARY KEY,
    id_producto INTEGER,
    id_cliente INTEGER,
    cantidad INTEGER,
    fecha DATE,
    total REAL,
    FOREIGN KEY (id_producto) REFERENCES productos(id_producto),
    FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente)
)
''')

# Insertar datos de ejemplo
print("\n Insertando datos de ejemplo...")

# Categorías
categorias_data = [
    (1, 'Electrónica', 'Dispositivos electrónicos'),
    (2, 'Ropa', 'Prendas de vestir'),
    (3, 'Alimentos', 'Productos alimenticios'),
    (4, 'Hogar', 'Artículos para el hogar')
]
cursor.executemany('INSERT INTO categorias VALUES (?, ?, ?)', categorias_data)

# Productos
productos_data = [
    (1, 'Laptop HP', 899.99, 1, 15),
    (2, 'Mouse Logitech', 29.99, 1, 50),
    (3, 'Teclado Mecánico', 79.99, 1, 30),
    (4, 'Camiseta Nike', 39.99, 2, 100),
    (5, 'Jeans Levis', 69.99, 2, 45),
    (6, 'Café Premium', 15.99, 3, 200),
    (7, 'Cereal Integral', 8.99, 3, 150),
    (8, 'Lámpara LED', 45.99, 4, 60),
    (9, 'Almohada Memory Foam', 34.99, 4, 80),
    (10, 'Monitor Samsung', 299.99, 1, 25)
]
cursor.executemany('INSERT INTO productos VALUES (?, ?, ?, ?, ?)', productos_data)

# Clientes
clientes_data = [
    (1, 'Juan Pérez', 'juan@email.com', 'Madrid', 'España'),
    (2, 'María García', 'maria@email.com', 'Barcelona', 'España'),
    (3, 'Carlos López', 'carlos@email.com', 'México DF', 'México'),
    (4, 'Ana Martínez', 'ana@email.com', 'Buenos Aires', 'Argentina'),
    (5, 'Luis Rodríguez', 'luis@email.com', 'Lima', 'Perú'),
    (6, 'Sofia Torres', 'sofia@email.com', 'Bogotá', 'Colombia'),
    (7, 'Pedro Sánchez', 'pedro@email.com', 'Santiago', 'Chile'),
    (8, 'Laura Fernández', 'laura@email.com', 'Montevideo', 'Uruguay')
]
cursor.executemany('INSERT INTO clientes VALUES (?, ?, ?, ?, ?)', clientes_data)

# Ventas
np.random.seed(42)
ventas_data = []
for i in range(1, 51):
    id_producto = np.random.randint(1, 11)
    id_cliente = np.random.randint(1, 9)
    cantidad = np.random.randint(1, 5)
    # Obtener precio del producto
    cursor.execute('SELECT precio FROM productos WHERE id_producto = ?', (id_producto,))
    precio = cursor.fetchone()[0]
    total = precio * cantidad
    fecha = f'2023-{np.random.randint(1, 13):02d}-{np.random.randint(1, 29):02d}'
    ventas_data.append((i, id_producto, id_cliente, cantidad, fecha, total))

cursor.executemany('INSERT INTO ventas VALUES (?, ?, ?, ?, ?, ?)', ventas_data)

conn.commit()

print("✓ Base de datos creada exitosamente")
print(f"  - {len(categorias_data)} categorías")
print(f"  - {len(productos_data)} productos")
print(f"  - {len(clientes_data)} clientes")
print(f"  - {len(ventas_data)} ventas")

                      CREACIÓN DE BASE DE DATOS                       

 Insertando datos de ejemplo...
✓ Base de datos creada exitosamente
  - 4 categorías
  - 10 productos
  - 8 clientes
  - 50 ventas


## Sintaxis SQL Básica

### ¿Qué es?

La **sintaxis SQL básica** es el conjunto de comandos fundamentales que permiten interactuar con bases de datos relacionales. Los tres comandos esenciales son SELECT (para seleccionar datos), FROM (para especificar la tabla), y WHERE (para filtrar resultados). Juntos forman la estructura básica de cualquier consulta SQL para extraer información de una base de datos.

### ¿Para qué sirve?

La sintaxis SQL básica es fundamental para:

- **Extraer datos específicos** de tablas usando SELECT con columnas particulares
- **Consultar información** de una o múltiples tablas usando FROM
- **Filtrar resultados** según condiciones específicas con WHERE
- **Realizar búsquedas precisas** usando operadores de comparación (=, >, <, >=, <=, !=)
- **Combinar condiciones** con operadores lógicos (AND, OR, NOT)
- **Buscar patrones** en texto usando LIKE
- **Validar pertenencia** a conjuntos de valores con IN
- **Realizar análisis de datos** extrayendo subconjuntos relevantes de información

En el análisis de datos, dominar estos comandos permite explorar datasets, identificar tendencias y preparar información para análisis más complejos.

### ¿Cómo se usa?

Las consultas SQL siguen la estructura básica: `SELECT columnas FROM tabla WHERE condición`. Se pueden combinar múltiples condiciones usando AND/OR, buscar patrones con LIKE, y verificar valores dentro de listas con IN. A continuación, se presentan ejemplos prácticos de cada técnica.

In [4]:
print("=" * 70)
print("SINTAXIS SQL BÁSICA".center(70))
print("=" * 70)

# 1. SELECT básico - Seleccionar todas las columnas
print("\n1. SELECT * - Todas las columnas")
print("-" * 70)
query = "SELECT * FROM productos LIMIT 5"
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 2. SELECT específico - Columnas seleccionadas
print("\n2. SELECT columnas específicas")
print("-" * 70)
query = "SELECT nombre_producto, precio FROM productos"
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 3. WHERE - Filtrado básico
print("\n3. WHERE - Filtrado por condición")
print("-" * 70)
query = "SELECT nombre_producto, precio FROM productos WHERE precio > 50"
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 4. WHERE con múltiples condiciones
print("\n4. WHERE - Múltiples condiciones (AND/OR)")
print("-" * 70)
query = """SELECT nombre_producto, precio, stock 
           FROM productos 
           WHERE precio > 30 AND stock < 100"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 5. WHERE con IN
print("\n5. WHERE - Operador IN")
print("-" * 70)
query = """SELECT nombre, ciudad FROM clientes 
           WHERE pais IN ('España', 'México')"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 6. WHERE con LIKE (patrón)
print("\n6. WHERE - Operador LIKE (búsqueda de patrones)")
print("-" * 70)
query = "SELECT nombre_producto, precio FROM productos WHERE nombre_producto LIKE '%Laptop%'"
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

print("\n OPERADORES DE COMPARACIÓN EN WHERE:")
print("  = (igual), != o <> (diferente), > (mayor), < (menor)")
print("  >= (mayor o igual), <= (menor o igual)")
print("  BETWEEN (rango), IN (lista), LIKE (patrón)")
print("  AND, OR, NOT (operadores lógicos)")

                         SINTAXIS SQL BÁSICA                          

1. SELECT * - Todas las columnas
----------------------------------------------------------------------
Query: SELECT * FROM productos LIMIT 5

   id_producto   nombre_producto  precio  id_categoria  stock
0            1         Laptop HP  899.99             1     15
1            2    Mouse Logitech   29.99             1     50
2            3  Teclado Mecánico   79.99             1     30
3            4     Camiseta Nike   39.99             2    100
4            5       Jeans Levis   69.99             2     45

2. SELECT columnas específicas
----------------------------------------------------------------------
Query: SELECT nombre_producto, precio FROM productos

        nombre_producto  precio
0             Laptop HP  899.99
1        Mouse Logitech   29.99
2      Teclado Mecánico   79.99
3         Camiseta Nike   39.99
4           Jeans Levis   69.99
5          Café Premium   15.99
6       Cereal Integral    8.99

## Filtrado y Ordenamiento

### ¿Qué es?

El **filtrado y ordenamiento** en SQL son técnicas para organizar y limitar los resultados de consultas. ORDER BY permite ordenar los datos ascendente (ASC) o descendentemente (DESC) según una o varias columnas, mientras que LIMIT restringe la cantidad de registros devueltos. Estas herramientas son esenciales para presentar datos de manera estructurada y manejar grandes volúmenes de información.

### ¿Para qué sirve?

El filtrado y ordenamiento sirven para:

- **Ordenar resultados** alfabéticamente o numéricamente con ORDER BY
- **Identificar valores extremos** (máximos/mínimos) ordenando de forma descendente o ascendente
- **Organizar por múltiples criterios** usando ORDER BY con varias columnas
- **Limitar resultados** a los primeros N registros con LIMIT
- **Implementar paginación** combinando LIMIT con OFFSET
- **Crear rankings** (top 10, top 5, etc.) ordenando y limitando resultados
- **Mejorar el rendimiento** de consultas limitando la cantidad de datos devueltos
- **Presentar información de manera clara** a usuarios finales

En análisis de datos, estas técnicas son fundamentales para crear reportes, identificar outliers y trabajar eficientemente con grandes datasets.

### ¿Cómo se usa?

ORDER BY se coloca después de WHERE (si existe) seguido del nombre de la columna y ASC/DESC. LIMIT se añade al final de la consulta con el número de registros deseados, opcionalmente seguido de OFFSET para paginación. El orden de ejecución es: WHERE → ORDER BY → LIMIT.

In [5]:
print("=" * 70)
print("FILTRADO Y ORDENAMIENTO".center(70))
print("=" * 70)

# 1. ORDER BY - Orden ascendente
print("\n1. ORDER BY ASC - Orden ascendente")
print("-" * 70)
query = "SELECT nombre_producto, precio FROM productos ORDER BY precio ASC"
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 2. ORDER BY - Orden descendente
print("\n2. ORDER BY DESC - Orden descendente")
print("-" * 70)
query = "SELECT nombre_producto, precio FROM productos ORDER BY precio DESC"
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 3. ORDER BY múltiples columnas
print("\n3. ORDER BY - Múltiples columnas")
print("-" * 70)
query = """SELECT nombre_producto, precio, stock 
           FROM productos 
           ORDER BY precio DESC, stock ASC"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 4. LIMIT - Limitar resultados
print("\n4. LIMIT - Primeros N resultados")
print("-" * 70)
query = """SELECT nombre_producto, precio 
           FROM productos 
           ORDER BY precio DESC 
           LIMIT 3"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)
print("\n→ Los 3 productos más caros")

# 5. LIMIT con OFFSET
print("\n5. LIMIT con OFFSET - Paginación")
print("-" * 70)
query = """SELECT nombre_producto, precio 
           FROM productos 
           ORDER BY precio DESC 
           LIMIT 3 OFFSET 3"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)
print("\n→ Productos del 4to al 6to (paginación)")

# 6. Combinación completa
print("\n6. Combinación: WHERE + ORDER BY + LIMIT")
print("-" * 70)
query = """SELECT nombre_producto, precio, stock 
           FROM productos 
           WHERE stock > 20 
           ORDER BY precio DESC 
           LIMIT 5"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

print("\n ORDEN DE EJECUCIÓN SQL:")
print("  1. FROM - Seleccionar tabla(s)")
print("  2. WHERE - Filtrar filas")
print("  3. SELECT - Seleccionar columnas")
print("  4. ORDER BY - Ordenar resultados")
print("  5. LIMIT - Limitar cantidad de resultados")

                       FILTRADO Y ORDENAMIENTO                        

1. ORDER BY ASC - Orden ascendente
----------------------------------------------------------------------
Query: SELECT nombre_producto, precio FROM productos ORDER BY precio ASC

        nombre_producto  precio
0       Cereal Integral    8.99
1          Café Premium   15.99
2        Mouse Logitech   29.99
3  Almohada Memory Foam   34.99
4         Camiseta Nike   39.99
5           Lámpara LED   45.99
6           Jeans Levis   69.99
7      Teclado Mecánico   79.99
8       Monitor Samsung  299.99
9             Laptop HP  899.99

2. ORDER BY DESC - Orden descendente
----------------------------------------------------------------------
Query: SELECT nombre_producto, precio FROM productos ORDER BY precio DESC

        nombre_producto  precio
0             Laptop HP  899.99
1       Monitor Samsung  299.99
2      Teclado Mecánico   79.99
3           Jeans Levis   69.99
4           Lámpara LED   45.99
5         Camiseta N

## Funciones de Agregación

### ¿Qué es?

Las **funciones de agregación** son operaciones que procesan múltiples filas de datos para producir un único valor de resumen. Las cinco funciones principales son: COUNT (contar registros), SUM (sumar valores), AVG (calcular promedio), MIN (valor mínimo) y MAX (valor máximo). Estas funciones son fundamentales para obtener estadísticas descriptivas directamente desde la base de datos.

### ¿Para qué sirve?

Las funciones de agregación sirven para:

- **COUNT**: Contar total de registros, contar valores no nulos, o contar valores únicos con DISTINCT
- **SUM**: Calcular totales de ventas, ingresos, cantidades o cualquier suma acumulativa
- **AVG**: Obtener promedios de precios, calificaciones, o métricas de rendimiento
- **MIN/MAX**: Identificar valores mínimos y máximos en rangos de datos
- **Generar KPIs** y métricas de negocio directamente desde SQL
- **Realizar análisis estadístico** sin necesidad de extraer todos los datos
- **Combinar múltiples agregaciones** en una sola consulta para dashboards
- **Filtrar datos antes de agregar** usando WHERE para análisis segmentados

En análisis de datos, estas funciones permiten obtener insights rápidos y crear reportes ejecutivos de manera eficiente.

### ¿Cómo se usa?

Las funciones de agregación se colocan en la cláusula SELECT: `SELECT COUNT(*), SUM(columna), AVG(columna) FROM tabla`. Se pueden combinar múltiples funciones en una consulta, usar DISTINCT para valores únicos, y filtrar antes de agregar con WHERE. El resultado es siempre un único valor (o una fila) por función aplicada.

In [6]:
print("=" * 70)
print("FUNCIONES DE AGREGACIÓN".center(70))
print("=" * 70)

# 1. COUNT - Contar registros
print("\n1. COUNT - Contar registros")
print("-" * 70)
query = "SELECT COUNT(*) as total_productos FROM productos"
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 2. SUM - Suma de valores
print("\n2. SUM - Suma total")
print("-" * 70)
query = "SELECT SUM(total) as ingresos_totales FROM ventas"
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 3. AVG - Promedio
print("\n3. AVG - Promedio")
print("-" * 70)
query = "SELECT AVG(precio) as precio_promedio FROM productos"
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 4. MIN y MAX
print("\n4. MIN y MAX - Valores mínimo y máximo")
print("-" * 70)
query = """SELECT 
           MIN(precio) as precio_minimo, 
           MAX(precio) as precio_maximo 
           FROM productos"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 5. Múltiples agregaciones
print("\n5. Múltiples funciones de agregación")
print("-" * 70)
query = """SELECT 
           COUNT(*) as total_ventas,
           SUM(total) as ingresos_totales,
           AVG(total) as ticket_promedio,
           MIN(total) as venta_minima,
           MAX(total) as venta_maxima
           FROM ventas"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 6. COUNT DISTINCT - Contar valores únicos
print("\n6. COUNT DISTINCT - Valores únicos")
print("-" * 70)
query = """SELECT 
           COUNT(DISTINCT id_cliente) as clientes_unicos,
           COUNT(DISTINCT id_producto) as productos_vendidos
           FROM ventas"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 7. Agregación con filtros
print("\n7. Agregación con WHERE")
print("-" * 70)
query = """SELECT COUNT(*) as ventas_grandes
           FROM ventas 
           WHERE total > 100"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

print("\n FUNCIONES DE AGREGACIÓN:")
print("  • COUNT(*): cuenta todas las filas")
print("  • COUNT(columna): cuenta valores no NULL")
print("  • COUNT(DISTINCT columna): cuenta valores únicos")
print("  • SUM(columna): suma de valores numéricos")
print("  • AVG(columna): promedio de valores")
print("  • MIN/MAX(columna): valor mínimo/máximo")

                       FUNCIONES DE AGREGACIÓN                        

1. COUNT - Contar registros
----------------------------------------------------------------------
Query: SELECT COUNT(*) as total_productos FROM productos

   total_productos
0               10

2. SUM - Suma total
----------------------------------------------------------------------
Query: SELECT SUM(total) as ingresos_totales FROM ventas

   ingresos_totales
0          18913.66

3. AVG - Promedio
----------------------------------------------------------------------
Query: SELECT AVG(precio) as precio_promedio FROM productos

   precio_promedio
0           152.59

4. MIN y MAX - Valores mínimo y máximo
----------------------------------------------------------------------
Query: SELECT 
           MIN(precio) as precio_minimo, 
           MAX(precio) as precio_maximo 
           FROM productos

   precio_minimo  precio_maximo
0           8.99         899.99

5. Múltiples funciones de agregación
----------------

## Agrupación

### ¿Qué es?

La **agrupación** en SQL es el proceso de organizar filas de datos en grupos basados en valores comunes de una o más columnas, usando la cláusula GROUP BY. HAVING complementa a GROUP BY permitiendo filtrar grupos después de la agregación (similar a WHERE pero para grupos). Esta combinación permite análisis segmentados y agregaciones por categorías.

### ¿Para qué sirve?

La agrupación sirve para:

- **Segmentar datos** por categorías, regiones, períodos de tiempo u otras dimensiones
- **Calcular métricas por grupo** (ventas por producto, ingresos por cliente, etc.)
- **Realizar análisis temporal** agrupando por fechas (día, mes, año)
- **Identificar patrones** en subconjuntos de datos
- **Filtrar grupos específicos** usando HAVING basado en resultados de agregación
- **Crear reportes segmentados** con múltiples niveles de agrupación
- **Combinar con funciones de agregación** para obtener estadísticas por categoría
- **Implementar análisis de cohortes** y segmentación de clientes

En análisis de datos, GROUP BY es esencial para análisis multidimensional, reportes por categoría y segmentación de información.

### ¿Cómo se usa?

GROUP BY se coloca después de WHERE y antes de ORDER BY: `SELECT columna, COUNT(*) FROM tabla GROUP BY columna`. HAVING se añade después de GROUP BY para filtrar grupos: `HAVING COUNT(*) > 5`. La diferencia clave: WHERE filtra filas antes de agrupar, HAVING filtra grupos después de agregar. Orden de ejecución: FROM → WHERE → GROUP BY → HAVING → SELECT → ORDER BY.

In [7]:
print("=" * 70)
print("GROUP BY Y HAVING".center(70))
print("=" * 70)

# 1. GROUP BY básico
print("\n1. GROUP BY - Agrupar y contar")
print("-" * 70)
query = """SELECT id_categoria, COUNT(*) as total_productos
           FROM productos
           GROUP BY id_categoria"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 2. GROUP BY con múltiples agregaciones
print("\n2. GROUP BY - Múltiples agregaciones")
print("-" * 70)
query = """SELECT 
           id_producto,
           COUNT(*) as num_ventas,
           SUM(cantidad) as unidades_vendidas,
           SUM(total) as ingresos_totales,
           AVG(total) as promedio_venta
           FROM ventas
           GROUP BY id_producto
           ORDER BY ingresos_totales DESC
           LIMIT 5"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 3. GROUP BY por fecha (análisis temporal)
print("\n3. GROUP BY - Por mes (análisis temporal)")
print("-" * 70)
query = """SELECT 
           strftime('%Y-%m', fecha) as mes,
           COUNT(*) as num_ventas,
           SUM(total) as ingresos
           FROM ventas
           GROUP BY strftime('%Y-%m', fecha)
           ORDER BY mes"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 4. HAVING - Filtrar grupos
print("\n4. HAVING - Filtrar resultados agrupados")
print("-" * 70)
query = """SELECT 
           id_cliente,
           COUNT(*) as num_compras,
           SUM(total) as total_gastado
           FROM ventas
           GROUP BY id_cliente
           HAVING COUNT(*) > 3
           ORDER BY total_gastado DESC"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)
print("\n→ Clientes con más de 3 compras")

# 5. HAVING con múltiples condiciones
print("\n5. HAVING - Múltiples condiciones")
print("-" * 70)
query = """SELECT 
           id_producto,
           COUNT(*) as num_ventas,
           AVG(cantidad) as cantidad_promedio
           FROM ventas
           GROUP BY id_producto
           HAVING COUNT(*) >= 2 AND AVG(cantidad) > 1.5
           ORDER BY num_ventas DESC"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 6. Combinación completa
print("\n6. WHERE + GROUP BY + HAVING + ORDER BY")
print("-" * 70)
query = """SELECT 
           id_producto,
           COUNT(*) as ventas,
           SUM(total) as ingresos
           FROM ventas
           WHERE total > 30
           GROUP BY id_producto
           HAVING COUNT(*) >= 2
           ORDER BY ingresos DESC
           LIMIT 5"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

print("\n DIFERENCIA ENTRE WHERE Y HAVING:")
print("  • WHERE: filtra filas ANTES de agrupar")
print("  • HAVING: filtra grupos DESPUÉS de agrupar")
print("\n  Orden de ejecución:")
print("  FROM → WHERE → GROUP BY → HAVING → SELECT → ORDER BY → LIMIT")

                          GROUP BY Y HAVING                           

1. GROUP BY - Agrupar y contar
----------------------------------------------------------------------
Query: SELECT id_categoria, COUNT(*) as total_productos
           FROM productos
           GROUP BY id_categoria

   id_categoria  total_productos
0             1                4
1             2                2
2             3                2
3             4                2

2. GROUP BY - Múltiples agregaciones
----------------------------------------------------------------------
Query: SELECT 
           id_producto,
           COUNT(*) as num_ventas,
           SUM(cantidad) as unidades_vendidas,
           SUM(total) as ingresos_totales,
           AVG(total) as promedio_venta
           FROM ventas
           GROUP BY id_producto
           ORDER BY ingresos_totales DESC
           LIMIT 5

   id_producto  num_ventas  unidades_vendidas  ingresos_totales  \
0            1           5                 12   

## Joins

### ¿Qué es?

Los **Joins** son operaciones que combinan filas de dos o más tablas basándose en una columna relacionada entre ellas. Los tipos principales son: INNER JOIN (solo registros coincidentes), LEFT JOIN (todos de la izquierda más coincidentes), RIGHT JOIN (todos de la derecha más coincidentes), y FULL OUTER JOIN (todos los registros de ambas tablas). Son fundamentales para trabajar con bases de datos relacionales normalizadas.

### ¿Para qué sirve?

Los Joins sirven para:

- **INNER JOIN**: Combinar tablas obteniendo solo registros que existen en ambas
- **LEFT JOIN**: Mantener todos los registros de la tabla principal incluso sin coincidencias
- **Relacionar información distribuida** en múltiples tablas (clientes, ventas, productos)
- **Enriquecer datos** añadiendo información de tablas relacionadas
- **Realizar análisis complejos** que requieren datos de múltiples fuentes
- **Identificar registros sin relación** (productos sin ventas, clientes sin compras)
- **Crear vistas consolidadas** de información para reportes
- **Combinar múltiples Joins** para consultas con 3 o más tablas relacionadas

En análisis de datos, los Joins permiten trabajar con modelos de datos normalizados y crear análisis completos combinando información de diferentes entidades.

### ¿Cómo se usa?

Los Joins se escriben en la cláusula FROM: `FROM tabla1 INNER JOIN tabla2 ON tabla1.id = tabla2.id`. Se pueden encadenar múltiples Joins para combinar varias tablas. La condición ON especifica qué columnas relacionar. LEFT JOIN incluye NULLs cuando no hay coincidencia en la tabla derecha. Se pueden usar alias (tabla1 t1) para simplificar consultas complejas.

In [8]:
print("=" * 70)
print("JOINS - COMBINACIÓN DE TABLAS".center(70))
print("=" * 70)

# 1. INNER JOIN - Solo registros coincidentes
print("\n1. INNER JOIN - Registros coincidentes en ambas tablas")
print("-" * 70)
query = """SELECT 
           v.id_venta,
           p.nombre_producto,
           v.cantidad,
           v.total,
           v.fecha
           FROM ventas v
           INNER JOIN productos p ON v.id_producto = p.id_producto
           LIMIT 5"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 2. Múltiples INNER JOINs
print("\n2. Múltiples INNER JOINs")
print("-" * 70)
query = """SELECT 
           v.id_venta,
           c.nombre as cliente,
           p.nombre_producto,
           v.cantidad,
           v.total
           FROM ventas v
           INNER JOIN productos p ON v.id_producto = p.id_producto
           INNER JOIN clientes c ON v.id_cliente = c.id_cliente
           LIMIT 8"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 3. JOIN con agregación
print("\n3. JOIN con GROUP BY - Ventas por cliente")
print("-" * 70)
query = """SELECT 
           c.nombre,
           c.ciudad,
           COUNT(v.id_venta) as num_compras,
           SUM(v.total) as total_gastado
           FROM clientes c
           INNER JOIN ventas v ON c.id_cliente = v.id_cliente
           GROUP BY c.id_cliente
           ORDER BY total_gastado DESC"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 4. JOIN con categorías
print("\n4. JOIN - Productos con categorías")
print("-" * 70)
query = """SELECT 
           cat.nombre_categoria,
           COUNT(v.id_venta) as num_ventas,
           SUM(v.total) as ingresos
           FROM categorias cat
           INNER JOIN productos p ON cat.id_categoria = p.id_categoria
           INNER JOIN ventas v ON p.id_producto = v.id_producto
           GROUP BY cat.id_categoria
           ORDER BY ingresos DESC"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 5. LEFT JOIN - Todos de la tabla izquierda
print("\n5. LEFT JOIN - Todos los productos (vendidos o no)")
print("-" * 70)
query = """SELECT 
           p.nombre_producto,
           COUNT(v.id_venta) as num_ventas,
           COALESCE(SUM(v.total), 0) as ingresos
           FROM productos p
           LEFT JOIN ventas v ON p.id_producto = v.id_producto
           GROUP BY p.id_producto
           ORDER BY num_ventas DESC"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)
print("\n→ COALESCE reemplaza NULL por 0")

# 6. Consulta compleja con múltiples JOINs
print("\n6. Consulta compleja - Análisis detallado de ventas")
print("-" * 70)
query = """SELECT 
           c.nombre as cliente,
           c.pais,
           cat.nombre_categoria as categoria,
           p.nombre_producto as producto,
           v.cantidad,
           v.total,
           v.fecha
           FROM ventas v
           INNER JOIN clientes c ON v.id_cliente = c.id_cliente
           INNER JOIN productos p ON v.id_producto = p.id_producto
           INNER JOIN categorias cat ON p.id_categoria = cat.id_categoria
           WHERE v.total > 100
           ORDER BY v.total DESC
           LIMIT 10"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

print("\n TIPOS DE JOINS:")
print("  • INNER JOIN: Solo registros que coinciden en ambas tablas")
print("  • LEFT JOIN: Todos de la izquierda + coincidentes de la derecha")
print("  • RIGHT JOIN: Todos de la derecha + coincidentes de la izquierda")
print("  • FULL OUTER JOIN: Todos los registros de ambas tablas")
print("\n  Nota: SQLite no soporta RIGHT JOIN ni FULL OUTER JOIN directamente")

                    JOINS - COMBINACIÓN DE TABLAS                     

1. INNER JOIN - Registros coincidentes en ambas tablas
----------------------------------------------------------------------
Query: SELECT 
           v.id_venta,
           p.nombre_producto,
           v.cantidad,
           v.total,
           v.fecha
           FROM ventas v
           INNER JOIN productos p ON v.id_producto = p.id_producto
           LIMIT 5

   id_venta   nombre_producto  cantidad   total       fecha
0         1   Cereal Integral         1    8.99  2023-11-08
1         2       Jeans Levis         2  139.98  2023-03-23
2         3       Lámpara LED         4  183.96  2023-08-24
3         4  Teclado Mecánico         1   79.99  2023-02-24
4         5      Café Premium         4   63.96  2023-12-21

2. Múltiples INNER JOINs
----------------------------------------------------------------------
Query: SELECT 
           v.id_venta,
           c.nombre as cliente,
           p.nombre_producto,
   

## Subconsultas

### ¿Qué es?

Las **subconsultas** (o subqueries) son consultas SQL anidadas dentro de otra consulta principal. Pueden ubicarse en las cláusulas WHERE (para filtrado), SELECT (para cálculos), o FROM (como tablas derivadas). Las subconsultas correlacionadas hacen referencia a la consulta externa, mientras que las no correlacionadas son independientes. Son una alternativa poderosa a los Joins para ciertos tipos de análisis.

### ¿Para qué sirve?

Las subconsultas sirven para:

- **Filtrar con condiciones dinámicas** (valores mayores al promedio calculado)
- **Comparar contra agregaciones** sin usar GROUP BY en la consulta principal
- **Crear tablas temporales** en FROM para consultas complejas paso a paso
- **Calcular valores en SELECT** basados en otras tablas
- **Usar IN para filtrar** con listas de valores obtenidos dinámicamente
- **Verificar existencia** con EXISTS/NOT EXISTS de manera eficiente
- **Resolver problemas complejos** dividiéndolos en pasos más simples
- **Evitar Joins complicados** en ciertos escenarios específicos

En análisis de datos, las subconsultas permiten realizar cálculos comparativos, filtrados dinámicos y análisis en múltiples niveles de agregación.

### ¿Cómo se usa?

Las subconsultas se escriben entre paréntesis. En WHERE: `WHERE precio > (SELECT AVG(precio) FROM productos)`. En SELECT: `SELECT col, (SELECT COUNT(*) FROM tabla2) FROM tabla1`. En FROM: `FROM (SELECT ... FROM tabla) AS subconsulta`. EXISTS retorna verdadero si la subconsulta tiene resultados. Las subconsultas deben retornar un solo valor cuando se usan con operadores de comparación (=, >, <).

In [9]:
print("=" * 70)
print("SUBCONSULTAS (SUBQUERIES)".center(70))
print("=" * 70)

# 1. Subconsulta en WHERE
print("\n1. Subconsulta en WHERE - Productos con precio > promedio")
print("-" * 70)
query = """SELECT nombre_producto, precio
           FROM productos
           WHERE precio > (SELECT AVG(precio) FROM productos)
           ORDER BY precio DESC"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 2. Subconsulta con IN
print("\n2. Subconsulta con IN - Clientes que han comprado")
print("-" * 70)
query = """SELECT nombre, email, ciudad
           FROM clientes
           WHERE id_cliente IN (SELECT DISTINCT id_cliente FROM ventas)"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 3. Subconsulta en SELECT
print("\n3. Subconsulta en SELECT - Calcular diferencia con promedio")
print("-" * 70)
query = """SELECT 
           nombre_producto,
           precio,
           (SELECT AVG(precio) FROM productos) as precio_promedio,
           precio - (SELECT AVG(precio) FROM productos) as diferencia
           FROM productos
           ORDER BY diferencia DESC
           LIMIT 5"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 4. Subconsulta en FROM (tabla derivada)
print("\n4. Subconsulta en FROM - Tabla derivada")
print("-" * 70)
query = """SELECT 
           categoria,
           AVG(total_ventas) as promedio_ventas_categoria
           FROM (
               SELECT 
                   p.id_categoria as categoria,
                   SUM(v.total) as total_ventas
               FROM productos p
               INNER JOIN ventas v ON p.id_producto = v.id_producto
               GROUP BY p.id_categoria, v.id_venta
           )
           GROUP BY categoria"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 5. Subconsulta correlacionada
print("\n5. Subconsulta correlacionada - Productos más caros de su categoría")
print("-" * 70)
query = """SELECT p1.nombre_producto, p1.precio, p1.id_categoria
           FROM productos p1
           WHERE p1.precio = (
               SELECT MAX(p2.precio)
               FROM productos p2
               WHERE p2.id_categoria = p1.id_categoria
           )
           ORDER BY p1.id_categoria"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

# 6. EXISTS - Verificar existencia
print("\n6. EXISTS - Productos que se han vendido")
print("-" * 70)
query = """SELECT nombre_producto, precio
           FROM productos p
           WHERE EXISTS (
               SELECT 1 FROM ventas v 
               WHERE v.id_producto = p.id_producto
           )
           LIMIT 8"""
print(f"Query: {query}\n")
df = pd.read_sql_query(query, conn)
print(df)

print("\n TIPOS DE SUBCONSULTAS:")
print("  • En WHERE: filtrar basado en resultado de otra consulta")
print("  • En SELECT: calcular valores basados en otras tablas")
print("  • En FROM: usar resultado como tabla temporal")
print("  • Correlacionadas: referencia a tabla de query externa")
print("  • EXISTS/NOT EXISTS: verificar existencia de registros")

                      SUBCONSULTAS (SUBQUERIES)                       

1. Subconsulta en WHERE - Productos con precio > promedio
----------------------------------------------------------------------
Query: SELECT nombre_producto, precio
           FROM productos
           WHERE precio > (SELECT AVG(precio) FROM productos)
           ORDER BY precio DESC

   nombre_producto  precio
0        Laptop HP  899.99
1  Monitor Samsung  299.99

2. Subconsulta con IN - Clientes que han comprado
----------------------------------------------------------------------
Query: SELECT nombre, email, ciudad
           FROM clientes
           WHERE id_cliente IN (SELECT DISTINCT id_cliente FROM ventas)

            nombre             email        ciudad
0       Juan Pérez    juan@email.com        Madrid
1     María García   maria@email.com     Barcelona
2     Carlos López  carlos@email.com     México DF
3     Ana Martínez     ana@email.com  Buenos Aires
4   Luis Rodríguez    luis@email.com          Li

## Conexión Python-SQL

### ¿Qué es?

La **conexión Python-SQL** es la integración entre Python y bases de datos SQL usando librerías como sqlite3 (nativa para SQLite) y SQLAlchemy (ORM multiplataforma). Estas herramientas permiten ejecutar consultas SQL desde Python, manipular datos con Pandas, y realizar operaciones CRUD (crear, leer, actualizar, eliminar) directamente desde código Python, combinando el poder del análisis de datos con la gestión de bases de datos.

### ¿Para qué sirve?

La conexión Python-SQL sirve para:

- **sqlite3**: Trabajar con bases de datos SQLite de forma nativa y ligera en Python
- **SQLAlchemy**: Conectar con múltiples motores de bases de datos (PostgreSQL, MySQL, SQLite, etc.)
- **Ejecutar consultas SQL** desde scripts de Python y obtener resultados
- **Integrar con Pandas** usando `read_sql_query()` y `to_sql()` para análisis fluido
- **Automatizar procesos ETL** extrayendo, transformando y cargando datos
- **Crear pipelines de datos** que combinan SQL y manipulación de DataFrames
- **Usar consultas parametrizadas** para prevenir SQL injection y mejorar seguridad
- **Gestionar transacciones** con commit y rollback para integridad de datos
- **Exportar análisis** desde Python a bases de datos de producción

En análisis de datos, esta integración permite trabajar con datos almacenados en bases de datos sin salir del ecosistema de Python.

### ¿Cómo se usa?

Con sqlite3: crear conexión con `sqlite3.connect()`, ejecutar con `cursor.execute()`, y obtener resultados con `fetchone()`/`fetchall()`. Con Pandas: `pd.read_sql_query(query, conexion)` para leer y `df.to_sql(nombre_tabla, conexion)` para escribir. SQLAlchemy usa `create_engine()` para crear la conexión y `text()` para consultas parametrizadas. Siempre cerrar conexiones con `close()` o usar context managers (`with`).

In [10]:
print("=" * 70)
print("CONEXIÓN PYTHON-SQL CON SQLITE3".center(70))
print("=" * 70)

# 1. Método básico con sqlite3
print("\n1. SQLITE3 - Conexión y ejecución básica")
print("-" * 70)

# Crear conexión
conn_sqlite = sqlite3.connect('ventas.db')
cursor = conn_sqlite.cursor()

# Ejecutar consulta
cursor.execute('SELECT COUNT(*) as total FROM productos')
resultado = cursor.fetchone()
print(f"Total de productos: {resultado[0]}")

# Múltiples resultados
cursor.execute('SELECT nombre_producto, precio FROM productos LIMIT 5')
resultados = cursor.fetchall()
print("\nPrimeros 5 productos:")
for nombre, precio in resultados:
    print(f"  {nombre}: ${precio}")

# 2. Usar parámetros (evitar SQL injection)
print("\n2. SQLITE3 - Consultas parametrizadas (seguras)")
print("-" * 70)
precio_minimo = 50
cursor.execute('SELECT nombre_producto, precio FROM productos WHERE precio > ?', (precio_minimo,))
resultados = cursor.fetchall()
print(f"Productos con precio > ${precio_minimo}:")
for nombre, precio in resultados:
    print(f"  {nombre}: ${precio}")

# 3. Insertar datos desde Python
print("\n3. SQLITE3 - Insertar datos")
print("-" * 70)
nuevo_producto = (11, 'Tablet Android', 249.99, 1, 35)
cursor.execute('INSERT INTO productos VALUES (?, ?, ?, ?, ?)', nuevo_producto)
conn_sqlite.commit()
print("✓ Producto insertado correctamente")

# Verificar
cursor.execute('SELECT * FROM productos WHERE id_producto = 11')
print(cursor.fetchone())

# 4. Actualizar datos
print("\n4. SQLITE3 - Actualizar datos")
print("-" * 70)
cursor.execute('UPDATE productos SET stock = ? WHERE id_producto = ?', (40, 11))
conn_sqlite.commit()
print(f"✓ Stock actualizado. Filas afectadas: {cursor.rowcount}")

# 5. Usar con Pandas
print("\n5. PANDAS + SQLITE3 - Leer a DataFrame")
print("-" * 70)
query = 'SELECT * FROM productos WHERE precio > 50'
df = pd.read_sql_query(query, conn_sqlite)
print(df)

# 6. Escribir DataFrame a SQL
print("\n6. PANDAS - Escribir DataFrame a SQL")
print("-" * 70)
df_nuevo = pd.DataFrame({
    'id_producto': [12, 13],
    'nombre_producto': ['Mouse Inalámbrico', 'Hub USB'],
    'precio': [19.99, 24.99],
    'id_categoria': [1, 1],
    'stock': [100, 75]
})

df_nuevo.to_sql('productos', conn_sqlite, if_exists='append', index=False)
print("✓ DataFrame guardado en la base de datos")

# Verificar
df_verificar = pd.read_sql_query('SELECT * FROM productos WHERE id_producto IN (12, 13)', conn_sqlite)
print(df_verificar)

conn_sqlite.close()

print("\n SQLITE3 - MÉTODOS PRINCIPALES:")
print("  • cursor.execute(sql): ejecuta consulta")
print("  • cursor.fetchone(): devuelve una fila")
print("  • cursor.fetchall(): devuelve todas las filas")
print("  • cursor.fetchmany(n): devuelve n filas")
print("  • conn.commit(): confirma cambios")
print("  • conn.rollback(): revierte cambios")

                   CONEXIÓN PYTHON-SQL CON SQLITE3                    

1. SQLITE3 - Conexión y ejecución básica
----------------------------------------------------------------------
Total de productos: 10

Primeros 5 productos:
  Laptop HP: $899.99
  Mouse Logitech: $29.99
  Teclado Mecánico: $79.99
  Camiseta Nike: $39.99
  Jeans Levis: $69.99

2. SQLITE3 - Consultas parametrizadas (seguras)
----------------------------------------------------------------------
Productos con precio > $50:
  Laptop HP: $899.99
  Teclado Mecánico: $79.99
  Jeans Levis: $69.99
  Monitor Samsung: $299.99

3. SQLITE3 - Insertar datos
----------------------------------------------------------------------
✓ Producto insertado correctamente
(11, 'Tablet Android', 249.99, 1, 35)

4. SQLITE3 - Actualizar datos
----------------------------------------------------------------------
✓ Stock actualizado. Filas afectadas: 1

5. PANDAS + SQLITE3 - Leer a DataFrame
---------------------------------------------------

In [11]:
print("=" * 70)
print("CONEXIÓN PYTHON-SQL CON SQLALCHEMY".center(70))
print("=" * 70)

# 1. Crear engine de SQLAlchemy
print("\n1. SQLALCHEMY - Crear engine")
print("-" * 70)
engine = create_engine('sqlite:///ventas.db')
print(f"✓ Engine creado: {engine}")

# 2. Ejecutar consultas con SQLAlchemy
print("\n2. SQLALCHEMY - Ejecutar consultas")
print("-" * 70)
with engine.connect() as connection:
    result = connection.execute(text('SELECT COUNT(*) as total FROM clientes'))
    total = result.fetchone()[0]
    print(f"Total de clientes: {total}")

# 3. Pandas con SQLAlchemy
print("\n3. PANDAS + SQLALCHEMY - Leer datos")
print("-" * 70)
query = """SELECT 
           p.nombre_producto,
           COUNT(v.id_venta) as ventas,
           SUM(v.total) as ingresos
           FROM productos p
           LEFT JOIN ventas v ON p.id_producto = v.id_producto
           GROUP BY p.id_producto
           ORDER BY ingresos DESC
           LIMIT 5"""
df = pd.read_sql_query(query, engine)
print(df)

# 4. Escribir con SQLAlchemy
print("\n4. PANDAS + SQLALCHEMY - Escribir datos")
print("-" * 70)
df_analytics = pd.DataFrame({
    'metrica': ['Total Ventas', 'Ticket Promedio', 'Clientes Únicos'],
    'valor': [1234.56, 45.67, 8],
    'fecha_calculo': ['2023-12-31'] * 3
})

df_analytics.to_sql('metricas', engine, if_exists='replace', index=False)
print("✓ Tabla de métricas creada")

# Leer la tabla creada
df_leido = pd.read_sql_table('metricas', engine)
print("\nDatos guardados:")
print(df_leido)

# 5. Transacciones con SQLAlchemy
print("\n5. SQLALCHEMY - Transacciones")
print("-" * 70)
try:
    with engine.begin() as connection:
        # Múltiples operaciones en una transacción
        connection.execute(text("UPDATE productos SET stock = stock - 1 WHERE id_producto = 1"))
        connection.execute(text("UPDATE productos SET stock = stock - 1 WHERE id_producto = 2"))
        print("✓ Transacción completada (commit automático)")
except Exception as e:
    print(f"✗ Error en transacción (rollback automático): {e}")

# 6. Consultas parametrizadas con SQLAlchemy
print("\n6. SQLALCHEMY - Consultas parametrizadas")
print("-" * 70)
precio_min = 100
query_param = text("SELECT nombre_producto, precio FROM productos WHERE precio > :precio_minimo")

with engine.connect() as connection:
    result = connection.execute(query_param, {"precio_minimo": precio_min})
    print(f"Productos con precio > ${precio_min}:")
    for row in result:
        print(f"  {row[0]}: ${row[1]}")

print("\n SQLALCHEMY VS SQLITE3:")
print("  ✓ SQLAlchemy: más abstracto, múltiples BD, ORM disponible")
print("  ✓ sqlite3: más ligero, específico para SQLite, nativo en Python")
print("  ✓ Ambos se integran perfectamente con Pandas")

                  CONEXIÓN PYTHON-SQL CON SQLALCHEMY                  

1. SQLALCHEMY - Crear engine
----------------------------------------------------------------------
✓ Engine creado: Engine(sqlite:///ventas.db)

2. SQLALCHEMY - Ejecutar consultas
----------------------------------------------------------------------
Total de clientes: 8

3. PANDAS + SQLALCHEMY - Leer datos
----------------------------------------------------------------------
    nombre_producto  ventas  ingresos
0         Laptop HP       5  10799.88
1   Monitor Samsung       4   3599.88
2  Teclado Mecánico       8   1439.82
3       Lámpara LED       6    965.79
4       Jeans Levis       4    769.89

4. PANDAS + SQLALCHEMY - Escribir datos
----------------------------------------------------------------------
✓ Tabla de métricas creada

Datos guardados:
           metrica    valor fecha_calculo
0     Total Ventas  1234.56    2023-12-31
1  Ticket Promedio    45.67    2023-12-31
2  Clientes Únicos     8.00    2023-

## Optimización de Consultas

### ¿Qué es?

La **optimización de consultas SQL** es el conjunto de técnicas y mejores prácticas para mejorar el rendimiento de las consultas a bases de datos. Incluye el uso de índices, análisis con EXPLAIN QUERY PLAN, selección específica de columnas, y elección de operadores eficientes. El objetivo es reducir el tiempo de ejecución y el uso de recursos, especialmente en bases de datos grandes.

### ¿Para qué sirve?

La optimización de consultas sirve para:

- **Crear índices** en columnas frecuentemente consultadas para búsquedas más rápidas
- **Analizar planes de ejecución** con EXPLAIN QUERY PLAN para identificar cuellos de botella
- **Seleccionar columnas específicas** en lugar de SELECT * para reducir transferencia de datos
- **Implementar paginación** con LIMIT y OFFSET para manejar grandes resultados
- **Usar EXISTS en lugar de IN** para subconsultas más eficientes
- **Filtrar antes de unir** aplicando WHERE antes de JOIN cuando sea posible
- **Reducir tiempo de respuesta** de aplicaciones que consultan bases de datos
- **Escalar aplicaciones** permitiendo manejar mayor volumen de datos y usuarios

En análisis de datos, la optimización es crucial para trabajar eficientemente con grandes datasets y reducir tiempos de espera en análisis exploratorios.

### ¿Cómo se usa?

Crear índices con `CREATE INDEX nombre ON tabla(columna)`. Analizar con `EXPLAIN QUERY PLAN SELECT ...` para ver cómo se ejecuta la consulta. Usar SELECT con columnas específicas en lugar de *. Implementar paginación: `LIMIT n OFFSET m`. Preferir EXISTS sobre IN para grandes subconsultas. Aplicar WHERE antes de JOIN. Evitar funciones en WHERE que impidan uso de índices. Mantener estadísticas actualizadas con ANALYZE.

In [12]:
print("=" * 70)
print("OPTIMIZACIÓN DE CONSULTAS".center(70))
print("=" * 70)

import time

# 1. EXPLAIN QUERY PLAN
print("\n1. EXPLAIN QUERY PLAN - Analizar ejecución")
print("-" * 70)
cursor = conn.cursor()
query = "SELECT * FROM ventas WHERE id_cliente = 5"
cursor.execute(f"EXPLAIN QUERY PLAN {query}")
plan = cursor.fetchall()
print(f"Query: {query}\n")
print("Plan de ejecución:")
for row in plan:
    print(f"  {row}")

# 2. Crear índices
print("\n2. ÍNDICES - Mejorar performance de búsquedas")
print("-" * 70)

# Crear índice en columna frecuentemente consultada
cursor.execute('CREATE INDEX IF NOT EXISTS idx_ventas_cliente ON ventas(id_cliente)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_ventas_producto ON ventas(id_producto)')
cursor.execute('CREATE INDEX IF NOT EXISTS idx_ventas_fecha ON ventas(fecha)')
conn.commit()
print("✓ Índices creados")

# Ver plan después del índice
cursor.execute(f"EXPLAIN QUERY PLAN {query}")
plan_con_indice = cursor.fetchall()
print("\nPlan de ejecución CON índice:")
for row in plan_con_indice:
    print(f"  {row}")

# 3. Comparar performance
print("\n3. COMPARACIÓN - SELECT * vs columnas específicas")
print("-" * 70)

# Consulta con SELECT *
start = time.time()
for _ in range(100):
    cursor.execute('SELECT * FROM ventas')
    cursor.fetchall()
tiempo_select_all = time.time() - start

# Consulta con columnas específicas
start = time.time()
for _ in range(100):
    cursor.execute('SELECT id_venta, total FROM ventas')
    cursor.fetchall()
tiempo_select_especifico = time.time() - start

print(f"SELECT * ejecutado 100 veces: {tiempo_select_all:.4f} segundos")
print(f"SELECT columnas ejecutado 100 veces: {tiempo_select_especifico:.4f} segundos")
print(f"Mejora: {((tiempo_select_all - tiempo_select_especifico) / tiempo_select_all * 100):.1f}%")

# 4. LIMIT para grandes resultados
print("\n4. USO DE LIMIT - Paginación eficiente")
print("-" * 70)
page_size = 10
page = 1
offset = (page - 1) * page_size

query_paginada = f"SELECT * FROM ventas LIMIT {page_size} OFFSET {offset}"
print(f"Query paginada (página {page}): {query_paginada}")
df_page = pd.read_sql_query(query_paginada, conn)
print(f"\n✓ Devuelve solo {len(df_page)} registros en lugar de todos")

# 5. EXISTS vs IN para subconsultas
print("\n5. EXISTS vs IN - Eficiencia en subconsultas")
print("-" * 70)

# Con IN
start = time.time()
query_in = """SELECT * FROM productos 
              WHERE id_producto IN (SELECT id_producto FROM ventas)"""
for _ in range(50):
    cursor.execute(query_in)
    cursor.fetchall()
tiempo_in = time.time() - start

# Con EXISTS
start = time.time()
query_exists = """SELECT * FROM productos p
                  WHERE EXISTS (SELECT 1 FROM ventas v WHERE v.id_producto = p.id_producto)"""
for _ in range(50):
    cursor.execute(query_exists)
    cursor.fetchall()
tiempo_exists = time.time() - start

print(f"IN ejecutado 50 veces: {tiempo_in:.4f} segundos")
print(f"EXISTS ejecutado 50 veces: {tiempo_exists:.4f} segundos")
print(f"\n→ EXISTS suele ser más eficiente para grandes conjuntos de datos")

# 6. Ver índices creados
print("\n6. LISTAR ÍNDICES - Ver índices de una tabla")
print("-" * 70)
cursor.execute("SELECT name FROM sqlite_master WHERE type='index' AND tbl_name='ventas'")
indices = cursor.fetchall()
print("Índices en tabla 'ventas':")
for idx in indices:
    print(f"  • {idx[0]}")

print("\n MEJORES PRÁCTICAS DE OPTIMIZACIÓN:")
print("  1. Usar índices en columnas de búsqueda frecuente (WHERE, JOIN)")
print("  2. SELECT solo columnas necesarias, evitar SELECT *")
print("  3. Usar LIMIT para paginar grandes resultados")
print("  4. EXISTS mejor que IN para subconsultas grandes")
print("  5. Filtrar con WHERE antes de JOIN cuando sea posible")
print("  6. Usar transacciones para múltiples INSERTs")
print("  7. EXPLAIN QUERY PLAN para analizar consultas lentas")
print("  8. Normalizar base de datos apropiadamente")
print("  9. Evitar funciones en WHERE (impiden uso de índices)")
print("  10. Mantener estadísticas actualizadas (ANALYZE en SQLite)")

                      OPTIMIZACIÓN DE CONSULTAS                       

1. EXPLAIN QUERY PLAN - Analizar ejecución
----------------------------------------------------------------------
Query: SELECT * FROM ventas WHERE id_cliente = 5

Plan de ejecución:
  (2, 0, 0, 'SCAN ventas')

2. ÍNDICES - Mejorar performance de búsquedas
----------------------------------------------------------------------
✓ Índices creados

Plan de ejecución CON índice:
  (3, 0, 0, 'SEARCH ventas USING INDEX idx_ventas_cliente (id_cliente=?)')

3. COMPARACIÓN - SELECT * vs columnas específicas
----------------------------------------------------------------------
SELECT * ejecutado 100 veces: 0.0084 segundos
SELECT columnas ejecutado 100 veces: 0.0064 segundos
Mejora: 24.3%

4. USO DE LIMIT - Paginación eficiente
----------------------------------------------------------------------
Query paginada (página 1): SELECT * FROM ventas LIMIT 10 OFFSET 0

✓ Devuelve solo 10 registros en lugar de todos

5. EXISTS vs IN

## Ejercicio Práctico Integrador

### ¿Qué es?

Un **ejercicio práctico integrador** es una implementación completa que combina todos los conceptos SQL aprendidos en este notebook para resolver un caso de negocio realista. Este ejercicio integra SELECT, WHERE, JOINs, funciones de agregación, GROUP BY, subconsultas y conexión Python-SQL para crear un sistema de análisis de ventas funcional.

### ¿Para qué sirve?

Los ejercicios integradores son fundamentales para:

- **Consolidar conocimientos** aplicando múltiples técnicas SQL simultáneamente
- **Simular un análisis real** de datos de negocio con métricas y KPIs
- **Crear dashboards** con consultas que combinan JOINs, agregaciones y subconsultas
- **Generar insights** identificando productos top, clientes clave y tendencias temporales
- **Practicar análisis geográfico** segmentando datos por país y región
- **Exportar resultados** guardando reportes en nuevas tablas y archivos CSV
- **Prepararse para proyectos profesionales** de análisis de datos con SQL

Este ejercicio implementa un dashboard completo de ventas con KPIs principales, rankings de productos y clientes, análisis por categoría, tendencias temporales y distribución geográfica.

### ¿Cómo se usa?

Se ejecutan múltiples consultas SQL complejas que combinan JOINs de varias tablas con funciones de agregación y GROUP BY. Los resultados se cargan en DataFrames de Pandas para su presentación y se exportan a CSV y nuevas tablas SQL. A continuación se presenta el análisis completo.

In [13]:
print("=" * 70)
print("EJERCICIO INTEGRADOR: ANÁLISIS DE VENTAS CON SQL".center(70))
print("=" * 70)

# Dashboard de métricas clave
print("\n" + "=" * 70)
print("DASHBOARD DE VENTAS".center(70))
print("=" * 70)

# 1. KPIs principales
query_kpis = """SELECT 
    COUNT(DISTINCT id_venta) as total_ventas,
    COUNT(DISTINCT id_cliente) as clientes_activos,
    COUNT(DISTINCT id_producto) as productos_vendidos,
    SUM(total) as ingresos_totales,
    AVG(total) as ticket_promedio,
    MIN(fecha) as primera_venta,
    MAX(fecha) as ultima_venta
    FROM ventas"""

df_kpis = pd.read_sql_query(query_kpis, conn)
print("\n📊 KPIs PRINCIPALES:")
print("-" * 70)
for col in df_kpis.columns:
    valor = df_kpis[col].iloc[0]
    if isinstance(valor, float):
        print(f"  {col.replace('_', ' ').title()}: ${valor:,.2f}")
    else:
        print(f"  {col.replace('_', ' ').title()}: {valor}")

# 2. Top 5 productos más vendidos
query_top_productos = """SELECT 
    p.nombre_producto,
    cat.nombre_categoria,
    COUNT(v.id_venta) as num_ventas,
    SUM(v.cantidad) as unidades_vendidas,
    SUM(v.total) as ingresos
    FROM ventas v
    INNER JOIN productos p ON v.id_producto = p.id_producto
    INNER JOIN categorias cat ON p.id_categoria = cat.id_categoria
    GROUP BY v.id_producto
    ORDER BY ingresos DESC
    LIMIT 5"""

df_top_productos = pd.read_sql_query(query_top_productos, conn)
print("\n🏆 TOP 5 PRODUCTOS POR INGRESOS:")
print("-" * 70)
print(df_top_productos.to_string(index=False))

# 3. Top clientes
query_top_clientes = """SELECT 
    c.nombre,
    c.ciudad,
    c.pais,
    COUNT(v.id_venta) as num_compras,
    SUM(v.total) as total_gastado,
    AVG(v.total) as ticket_promedio
    FROM ventas v
    INNER JOIN clientes c ON v.id_cliente = c.id_cliente
    GROUP BY v.id_cliente
    ORDER BY total_gastado DESC
    LIMIT 5"""

df_top_clientes = pd.read_sql_query(query_top_clientes, conn)
print("\n👥 TOP 5 CLIENTES POR GASTO:")
print("-" * 70)
print(df_top_clientes.to_string(index=False))

# 4. Análisis por categoría
query_categorias = """SELECT 
    cat.nombre_categoria,
    COUNT(DISTINCT p.id_producto) as num_productos,
    COUNT(v.id_venta) as num_ventas,
    SUM(v.total) as ingresos,
    ROUND(SUM(v.total) * 100.0 / (SELECT SUM(total) FROM ventas), 2) as porcentaje_ingresos
    FROM categorias cat
    LEFT JOIN productos p ON cat.id_categoria = p.id_categoria
    LEFT JOIN ventas v ON p.id_producto = v.id_producto
    GROUP BY cat.id_categoria
    ORDER BY ingresos DESC"""

df_categorias = pd.read_sql_query(query_categorias, conn)
print("\n📦 ANÁLISIS POR CATEGORÍA:")
print("-" * 70)
print(df_categorias.to_string(index=False))

# 5. Análisis temporal
query_temporal = """SELECT 
    strftime('%Y-%m', fecha) as mes,
    COUNT(*) as num_ventas,
    SUM(cantidad) as unidades,
    SUM(total) as ingresos,
    AVG(total) as ticket_promedio
    FROM ventas
    GROUP BY strftime('%Y-%m', fecha)
    ORDER BY mes"""

df_temporal = pd.read_sql_query(query_temporal, conn)
print("\n📅 TENDENCIA MENSUAL:")
print("-" * 70)
print(df_temporal.to_string(index=False))

# 6. Análisis geográfico
query_geografico = """SELECT 
    c.pais,
    COUNT(DISTINCT c.id_cliente) as num_clientes,
    COUNT(v.id_venta) as num_ventas,
    SUM(v.total) as ingresos
    FROM clientes c
    INNER JOIN ventas v ON c.id_cliente = v.id_cliente
    GROUP BY c.pais
    ORDER BY ingresos DESC"""

df_geografico = pd.read_sql_query(query_geografico, conn)
print("\n🌍 DISTRIBUCIÓN GEOGRÁFICA:")
print("-" * 70)
print(df_geografico.to_string(index=False))

# 7. Productos sin ventas (oportunidad)
query_sin_ventas = """SELECT 
    p.nombre_producto,
    p.precio,
    p.stock,
    cat.nombre_categoria
    FROM productos p
    INNER JOIN categorias cat ON p.id_categoria = cat.id_categoria
    LEFT JOIN ventas v ON p.id_producto = v.id_producto
    WHERE v.id_venta IS NULL"""

df_sin_ventas = pd.read_sql_query(query_sin_ventas, conn)
if len(df_sin_ventas) > 0:
    print("\n⚠️ PRODUCTOS SIN VENTAS (Oportunidad de mejora):")
    print("-" * 70)
    print(df_sin_ventas.to_string(index=False))
else:
    print("\n✓ Todos los productos han tenido al menos una venta")

# 8. Guardar reporte en nueva tabla
print("\n💾 GUARDAR REPORTE:")
print("-" * 70)
cursor.execute('DROP TABLE IF EXISTS reporte_ventas')
df_top_productos.to_sql('reporte_ventas', conn, if_exists='replace', index=False)
print("✓ Reporte guardado en tabla 'reporte_ventas'")

# 9. Exportar a CSV
df_top_productos.to_csv('reporte_top_productos.csv', index=False)
df_top_clientes.to_csv('reporte_top_clientes.csv', index=False)
print("✓ Reportes exportados a CSV")

print("\n" + "=" * 70)
print("ANÁLISIS COMPLETADO".center(70))
print("=" * 70)
print("\n📋 ARCHIVOS GENERADOS:")
print("  • ventas.db (Base de datos SQLite)")
print("  • reporte_top_productos.csv")
print("  • reporte_top_clientes.csv")

print("\n💡 INSIGHTS CLAVE:")
print(f"  1. Total de ingresos: ${df_kpis['ingresos_totales'].iloc[0]:,.2f}")
print(f"  2. Ticket promedio: ${df_kpis['ticket_promedio'].iloc[0]:,.2f}")
print(f"  3. Producto top: {df_top_productos['nombre_producto'].iloc[0]}")
print(f"  4. Cliente top: {df_top_clientes['nombre'].iloc[0]}")
print(f"  5. País con más ingresos: {df_geografico['pais'].iloc[0]}")

# Cerrar conexión
conn.close()
print("\n✓ Conexión cerrada")

           EJERCICIO INTEGRADOR: ANÁLISIS DE VENTAS CON SQL           

                         DASHBOARD DE VENTAS                          

📊 KPIs PRINCIPALES:
----------------------------------------------------------------------
  Total Ventas: 50
  Clientes Activos: 8
  Productos Vendidos: 10
  Ingresos Totales: $18,913.66
  Ticket Promedio: $378.27
  Primera Venta: 2023-01-01
  Ultima Venta: 2023-12-25

🏆 TOP 5 PRODUCTOS POR INGRESOS:
----------------------------------------------------------------------
 nombre_producto nombre_categoria  num_ventas  unidades_vendidas  ingresos
       Laptop HP      Electrónica           5                 12  10799.88
 Monitor Samsung      Electrónica           4                 12   3599.88
Teclado Mecánico      Electrónica           8                 18   1439.82
     Lámpara LED            Hogar           6                 21    965.79
     Jeans Levis             Ropa           4                 11    769.89

👥 TOP 5 CLIENTES POR GASTO:
---

## Resumen y Recursos

### ¿Qué es?

El **resumen** es una recopilación de todos los comandos, técnicas y conceptos SQL cubiertos en este notebook. Funciona como una referencia rápida (cheat sheet) que consolida las herramientas fundamentales para interactuar con bases de datos relacionales desde Python, desde consultas básicas hasta optimización avanzada.

### ¿Para qué sirve?

Este resumen sirve para:

- **Consulta rápida** de sintaxis SQL cuando se necesita recordar un comando específico
- **Repaso general** de todos los temas cubiertos antes de un proyecto o entrevista
- **Referencia durante el trabajo** para verificar la estructura correcta de consultas
- **Identificar áreas de mejora** revisando qué conceptos necesitan más práctica
- **Guía de estudio** para reforzar los fundamentos de SQL y su integración con Python

### ¿Cómo se usa?

Utiliza esta sección como referencia rápida. Cada bloque agrupa los comandos por categoría para facilitar la búsqueda.

**Consultas básicas:**
- `SELECT`, `FROM`, `WHERE`, `ORDER BY`, `LIMIT`

**Agregación:**
- `COUNT()`, `SUM()`, `AVG()`, `MIN()`, `MAX()`
- `GROUP BY`, `HAVING`

**Joins:**
- `INNER JOIN`, `LEFT JOIN`, `RIGHT JOIN`, `FULL OUTER JOIN`

**Subconsultas:**
- En `WHERE`, en `SELECT`, en `FROM`
- `EXISTS`, `IN`

**Optimización:**
- Índices con `CREATE INDEX`
- `EXPLAIN QUERY PLAN`
- Mejores prácticas de performance

**Python-SQL:**
- `sqlite3`: conexión nativa
- `SQLAlchemy`: ORM y abstracción
- Integración con `Pandas`